In [ ]:
# Processor requirements
# The first two cells must be code cells, inpath and outpath define the scan file location and output location
# inpath = '/dls/i16/data/2025/mm41580-1/processed/1114510_msmapper.nxs'
inpath = '/dls/i16/data/2025/mm41580-1/1114510.nxs'
inpath = '/dls/i16/data/2025/mm41697-1/1116348.nxs'
outpath = ""

inpath = '/dls/science/groups/das/ExampleData/hdfmap_tests/i16/1116988.nxs'

# Plot msmapper volumetric plot
## Using PyVista

This notebook automatically loads remapped HKL volumes resulting from the MillerSpaceMapper software on I16 and plots the re-mapped volume as a volumetric plot using pyvista.

See https://confluence.diamond.ac.uk/display/I16/HKL+Mapping



## Re-running this notebook
It is possible to re-run this notebook using Jupyter notebook or Jupyter lab, however you need to be running in a python environment (or python kernal) that contains mmg_toolbox and the other python packages (numpy, matplotlib). You can install all the dependencies and start jupyter as follows (python 3.12+ required):
```bash
$ python -m pip install mmg_toolbox[full] pyvista[jupyter]
$ jupyter notebook /loc/of/this/notebook.ipynb
```

In [ ]:
import os
import time
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Markdown, Latex

from mmg_toolbox import data_file_reader, module_info
from mmg_toolbox import fitting
from mmg_toolbox.plotting.matplotlib import set_plot_defaults

set_plot_defaults()  # set custom matplotlib rcParams

def md(string):
    return display(Markdown(string))

print(module_info())

In [ ]:
# load scan file
scan = data_file_reader(inpath)

md(scan.format('# {beamline} Scan: {scan_number:.0f}\n*{filename}*'))

# Time
start = scan('start_time')
stop = scan('end_time')
duration = stop - start

# Create table
tab = '| field | metadata |\n| --- | --- |\n'
for s in str(scan).split('\n'):
    if '=' in s:
        tab += '| %s |\n' % s.replace('=', '|')

tab += '|**start time** | %s |\n' % start.strftime('%Y-%m-%d %H:%M')
tab += '|**end time** | %s |\n' % stop.strftime('%Y-%m-%d %H:%M')
tab += '|**duration** | %s |\n' % duration

md(tab)

In [ ]:
md('# Default Plot')
if 'volume' not in scan.map:
    scan.plot()

In [ ]:
# If scan is a standard nexus file, not a remapped file, look for the remapped file in the processing folder

if 'volume' in scan.map:
    remap = scan
    print('File contains remapped data')
else:
    # Find remapped files
    proc_dir = os.path.dirname(scan.filename) + '/processed/'
    scn = str(scan.scan_number())
    default = proc_dir + f"{scn}_msmapper.nxs"
    if os.path.isfile(default):
        remap = data_file_reader(default)
    else:
        for ntries in range(10):
            # remapping may take some time, so keep checking until finished
            files = [proc_dir + file for file in os.listdir(proc_dir)]

            mapper_files = [
                file for file in files
                if file.endswith('.nxs') and scn in file and
                   'volume' in data_file_reader(file).map
            ]
            if len(mapper_files) > 0:
                remap = data_file_reader(mapper_files[0])
                break
            else:
                print('Remapped file does not exist yet, try again in 1 min')
                time.sleep(60)

print('Remapped file loaded: %s' % remap.filename)

In [ ]:
# Get reciprocal space data from file
h, k, l, vol = remap('h_axis, k_axis, l_axis, volume')
# metadata
title = scan.format('#{scan_number:.0f}: {(cmd|user_command|scan_command)}')
hkl = scan.eval('array([h, k, l])')
vol_hkl = np.array([h.mean(), k.mean(), l.mean()])
pixel_size = remap('/entry0/instrument/pil3_100k/module/fast_pixel_direction')  # float, mm
detector_distance = remap('/entry0/instrument/pil3_100k/transformations/origin_offset')  # float, mm
# average angle subtended by each pixel
solid_angle = pixel_size ** 2 / detector_distance ** 2  # sr
vol = vol * solid_angle

md(remap.format("""
## metadata
 - scan hkl: ({mean(diffractometer_sample_h):.3f}, {mean(diffractometer_sample_k):.3f}, {mean(diffractometer_sample_l):.3f})
 - volume hkl: ({mean(h_axis):.3f}, {mean(k_axis):.3f}, {mean(l_axis):.3f})
 - pixel_size: {s_fast_pixel_direction}
 - detector_distance: {s_origin_offset}
## Reciprocal Space Volume
- h_axis: {s_h_axis}
- k_axis: {s_k_axis}
- l_axis: {s_l_axis}
- volume: {s_volume}
"""))

md(f'Each pixel is normalised by the solid angle: {solid_angle: .4g} sr\n\nThe max intensity is {vol.max():.4g} counts')


### Volume Statistics

In [ ]:
# Volume statistics
nonzero_vol = vol[vol > 0]
background = nonzero_vol.min() + 1
signal = nonzero_vol.max() - background
ii, ij, ik = fitting.max_index(vol)

In [ ]:
# Plot histogram
cmap = plt.get_cmap()
cut_ratios=(1e-3, 1e-2, 1e-1)  # lines at points relative to max intensity
alphas = np.linspace(0.1, 1, len(cut_ratios))
max_volume = vol.max()

ax = plt.figure().add_subplot()
n, bins, patches = ax.hist(np.log10(vol[vol > 0].flatten()), 100)
mode_background = 10 ** bins[np.argmax(n)]

for cut, alpha in zip(cut_ratios, alphas):
    logval = np.log10(cut * max_volume)
    ax.axvline(logval, marker='', c='k')
    for patch in patches:
        if patch.xy[0] >= logval:
            patch.set_color(cmap(alpha))

ax.axvline(np.log10(background), marker='', c='b')
ax.axvline(np.log10(mode_background), marker='', c='r')

ax.set_xlabel('Log$_{10}$ Voxel Intensity')
ax.set_ylabel('N')
ax.set_title(title)

## Convert to cartesian coordinates (Q)


In [ ]:
# Fail if hkl values are not consistent
assert np.linalg.norm(vol_hkl - hkl) < 0.5, f"Measured hkl={hkl} is different from remapped hkl={vol_hkl}"

In [ ]:
from mmg_toolbox.diffraction.lattice import bmatrix

# Convert to Q
a, b, c, alpha, beta, gamma = scan('unit_cell')
energy = scan('incident_energy')

# Build reciprocal lattice in orthogonal basis
astar, bstar, cstar = bmatrix(a, b, c, alpha, beta, gamma)
kk, hh, ll = np.meshgrid(k, h, l)
q = astar * hh.reshape(-1, 1) + bstar * kk.reshape(-1, 1) + cstar * ll.reshape(-1, 1)
qx = q[:, 0].reshape(hh.shape)
qy = q[:, 1].reshape(hh.shape)
qz = q[:, 2].reshape(hh.shape)
# magnitude of wavevector-transfer Q=kf-ki
qmag = np.sqrt(qx **2 + qy ** 2 + qz **2)

## PyVista 3D volumetric plot
See https://docs.pyvista.org/ 

In [ ]:
import pyvista as pv

# Example: Create volumetric data
grid = pv.ImageData()
grid.dimensions = vol.shape
grid.spacing = (h[1]-h[0], k[1]-k[0], l[1]-l[0])
grid.origin = (h[0], k[0], l[1])
grid.point_data["values"] = vol.flatten(order='F')

# Create a plotter
plotter = pv.Plotter()
actor = plotter.add_volume(grid, cmap="viridis", opacity="sigmoid", n_colors=3276)

# Add a faint border (bounding box)
plotter.show_bounds(
    color="silver",      # border color
    grid=True,
    show_xaxis=True,    # optional: hide labels/ticks
    show_yaxis=True,
    show_zaxis=True,
    xtitle='h',
    ytitle='k',
    ztitle='l',
)

# Add colorscale sliders
init_min, init_max = actor.mapper.scalar_range
clim = [np.log10(init_min), np.log10(init_max)]
print(f"clim: {clim}")
crange = [0, np.log10(vol.max())]

eps = 1e-12  # small gap to keep min < max

def set_cmin(v):
    # ensure cmin < cmax
    clim[0] = min(float(v), clim[1] - eps)
    actor.mapper.scalar_range = (10**clim[0], 10**clim[1])
    plotter.render()

def set_cmax(v):
    # ensure cmax > cmin
    clim[1] = max(float(v), clim[0] + eps)
    actor.mapper.scalar_range = (10**clim[0], 10**clim[1])
    plotter.render()

# Use the full data range as slider bounds
plotter.add_slider_widget(
    set_cmin,
    rng=crange,
    value=clim[0],
    title="Min color limit",
    pointa=(0.10, 0.90),
    pointb=(0.90, 0.90),
    style='modern',
)

plotter.add_slider_widget(
    set_cmax,
    rng=crange,
    value=clim[1],
    title="Max color limit",
    pointa=(0.10, 0.75),
    pointb=(0.90, 0.75),
    style='modern',
)

plotter.show()